- 동시성 제어는 데이터베이스에서 여러 트랜잭션이 동시에 실행될 때 ACID의 두 가지 속성인 원자성과 격리성을 유지하는 메커니즘

- 동시성 제어 기술 에는 MVCC( Multi-version Concurrency Control ), S2PL( Strict Two-Phase Locking ), OCC( Optimistic Concurrency Control ) 라는 세 가지 광범위한 동시성 제어 기술이 있음
  - 각 기술에는 다양한 변형이 있음

- MVCC에서 각 쓰기 작업은 이전 버전을 유지하면서 데이터 항목의 새 버전을 생성
  - 트랜잭션이 데이터 항목을 읽을 때 시스템은 개별 트랜잭션의 격리를 보장하기 위해 버전 중 하나를 선택
  - MVCC의 가장 큰 장점은 ' 리더가 작성자를 차단하지 않고, 작성자가 작성자를 차단하지 않는다 '는 점인데, 예를 들어 S2PL 기반 시스템은 작성자가 아이템을 쓸 때 작성자가 독점권을 획득하기 때문에 작성자를 차단해야 함
  - 항목에 대한 잠금을 설정
  - PostgreSQL과 일부 RDBMS는 SI(Snapshot Isolation) 라는 MVCC의 변형을 사용

- SI를 구현하기 위해 Oracle과 같은 일부 RDBMS는 롤백 세그먼트를 사용
  - 새 데이터 항목을 쓸 때 해당 항목의 이전 버전이 롤백 세그먼트에 기록되고 이후 새 항목이 데이터 영역에 덮어쓰여짐
  - PostgreSQL은 더 간단한 방법을 사용
  - 새로운 데이터 항목이 관련 테이블 페이지에 직접 삽입
  - 항목을 읽을 때 PostgreSQL은 가시성 확인 규칙을 적용하여 개별 트랜잭션에 대한 응답으로 항목의 적절한 버전을 선택

- SI는 ANSI SQL-92 표준에 정의된 세 가지 예외인 Dirty Reads , Non-Repeatable Reads 및 Phantom Reads를 허용하지 않음
  - 그러나 SI는 쓰기 왜곡(Write Skew) 및 읽기 전용 트랜잭션 왜곡(Read-only Transaction Skew) 과 같은 직렬화 예외를 허용하기 때문에 진정한 직렬화 가능성을 달성할 수 없음
  - 고전적인 직렬성 정의에 기반한 ANSI SQL-92 표준은 현대 이론의 정의와 동일 하지 않음

- 이 문제를 해결하기 위해 SSI(Serialized Snapshot Isolation)가 버전 9-1부터 추가
  - SSI는 직렬화 이상을 감지하고 이러한 이상으로 인해 발생하는 충돌을 해결할 수 있음
  - 따라서 PostgreSQL 버전 9-1 이상은 진정한 SERIALIZABLE 격리 수준을 제공
  - (또한 SQL Server도 SSI를 사용하지만 Oracle은 여전히 ​​SI만 사용)